# 데이터

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_Gen = ImageDataGenerator (
    rescale = 1./255, 
    horizontal_flip = True,         # 수평선을 기준으로 반전
    vertical_flip = True,           # 수직선을 기준으로 반전
    width_shift_range = 0.1,        # 10% 만큼 가로 이동
    height_shift_range = 0.1,       # 10% 만큼 세로 이동
    rotation_range = 5,             # 회전
    zoom_range = 0.7,               # 확대
    shear_range = 0.7,              # 엇갈림
    fill_mode = 'nearest'           # 축 이동을 했을 때 비는 부분을 가까운 값으로 채운다.
)

test_data_Gen = ImageDataGenerator (
    rescale = 1./255                # 테스트 데이터는 리스케일링만 한다.: 평가에 쓰이는 테스트 데이터는 증폭시키지 않은 원 데이터 사용
)

xy_train = train_data_Gen.flow_from_directory (
    'C:/study/keras/keras_data/brain/train',
    target_size=(100,100),
    batch_size= 1024,                  # 10개씩 자르므로 16번 돈다.
    class_mode='binary',
    color_mode='grayscale',
    shuffle=True
)

# ad(alzheimer disease) = 0, normal = 1  
# x = (160, 150, 150, 1), y = (160,) → 흑백 image 
# 160장 중에 ad, normal 각 80장씩
# np.unique = 0, 1

xy_test = train_data_Gen.flow_from_directory (
    'C:/study/keras/keras_data/brain/test',
    target_size=(100,100),
    batch_size=1024,                  # 10개씩 자르므로 16번 돈다.
    class_mode='binary',
    color_mode='grayscale',
    shuffle=True
)

# print(xy_train, '\n', xy_test)    # <keras.preprocessing.image.DirectoryIterator object at 0x000002862718CAC0> 
# print(xy_train[0])                # x data와 y data 개수, y data 개수 = batch_size 개수 만큼
# print(xy_train[0][0])
# print(xy_train[0][1])
print(len(xy_train[0]))  
print(xy_train[0][0].shape)         # x (10 = batch_size)
print(xy_train[0][1].shape)         # y

# batch_size를 전체 데이터보다 최대한 크게 끔 지정하면 전체 데이터의 개수를 알 수 있다.
print(type(xy_train)) 
print(type(xy_train[0]))             # 튜플은 변경 불가
print(type(xy_train[0][0]))          # x
print(type(xy_train[0][1]))          # y

Found 160 images belonging to 2 classes.
Found 120 images belonging to 2 classes.
2
(160, 100, 100, 1)
(160,)
<class 'keras.preprocessing.image.DirectoryIterator'>
<class 'tuple'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


 # 모델

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

model = Sequential()
model.add(Conv2D(64, (2,2), input_shape=(100, 100, 1), activation='relu'))
model.add(Conv2D(64, (2,2), activation='relu'))
model.add(Conv2D(64, (2,2), activation='relu'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

 # 컴파일, 훈련

In [3]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')
# hist = model.fit_generator(xy_train, steps_per_epoch=16, epochs=128, validation_data=xy_test, validation_steps=4)  # steps_per_epoch: 1 에포크 당 배치 사이즈에 따른 훈련 횟수
hist = model.fit(xy_train, epochs=128, batch_size=10, validation_data=(xy_test[0][0], xy_test[0][1]))  # steps_per_epoch: 1 에포크 당 배치 사이즈에 따른 훈련 횟수

acc = hist.history['acc']
val_acc = hist.history['val_acc']
loss = hist.history['loss']
val_loss = hist.history['val_loss']
print('acc: ', acc[-1])
print('loss: ', loss[-1])
print('val_acc: ', val_acc[-1])
print('val_loss: ', val_loss[-1])

C:\Users\bitcamp\AppData\Local\Temp\ipykernel_5736\1532313118.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(xy_train, steps_per_epoch=16, epochs=128, validation_data=xy_test, validation_steps=4)  # steps_per_epoch: 1 에포크 당 배치 사이즈에 따른 훈련 횟수


Epoch 1/128
16/16 [==============================] - 5s 87ms/step - loss: 0.6930 - acc: 0.4938 - val_loss: 1.0385 - val_acc: 0.5000
acc:  0.4937500059604645
loss:  0.6930215358734131
val_acc:  0.5
val_loss:  1.0385197401046753
